In [3]:
%cd /root/sagemaker/notebooks/

/root/sagemaker/notebooks


In [4]:
import datetime
import boto3
import pandas as pd
import sagemaker
from sagemaker import get_execution_role

BUCKET = 'sagemaker-churn-20220310'
PREFIX = 'mnist'
LOCAL_DATA_DIRECTORY = f'../data/{PREFIX}'
print(f"Artifacts will be written to s3://{BUCKET}/{PREFIX}")

sagemaker_session = sagemaker.Session()
boto_session = sagemaker_session.boto_session
sagemaker_client = boto_session.client('sagemaker')
role = get_execution_role()

Artifacts will be written to s3://sagemaker-churn-20220310/mnist


In [5]:
from smexperiments.experiment import Experiment

Experiment?

Init signature: Experiment(sagemaker_boto_client, **kwargs)
Docstring:     
An Amazon SageMaker experiment, which is a collection of related trials.

New experiments are created by calling :meth:`~smexperiments.experiment.Experiment.create`. Existing
experiments can be reloaded by calling :meth:`~smexperiments.experiment.Experiment.load`. You can
add a new trial to an Experiment by calling :meth:`~smexperiments.experiment.Experiment.create_trial`.
To remove an experiment and associated trials, trial components by calling :meth:`~smexperiments.experiment
.Experiment.delete_all`.

Examples:
    .. code-block:: python

        from smexperiments import experiment

        my_experiment = experiment.Experiment.create(experiment_name='AutoML')
        my_trial = my_experiment.create_trial(trial_name='random-forest')

        for exp in experiment.Experiment.list():
            print(exp)
        for trial in my_experiment.list_trials():
            print(trial)

        my_experiment.delete

In [7]:
from smexperiments.trial import Trial

Trial?

Init signature: Trial(sagemaker_boto_client, **kwargs)
Docstring:     
An execution of a data-science workflow with an experiment.

Consists of a list of trial component objects, which document individual activities within the workflow.

Examples:
    .. code-block:: python

        from smexperiments import trial, experiment, tracker

        my_experiment = experiment.Experiment.create(experiment_name='AutoML')
        my_trial = trial.Trial.create('AutoML')

        my_tracker = tracker.Tracker.create()
        # log hyper parameter of learning rate
        my_tracker.log_parameter('learning_rate', 0.01)
        my_trial.add_trial_component(my_tracker)

        for trial_component in my_trial.list_trial_components():
            print(trial_component)

        my_trial.remove_trial_component(my_tracker)
        my_trial.delete()

Attributes:
    trial_name (str): The name of the trial.
    experiment_name (str): The name of the trial's experiment.
    tags (List[dict[str, str]]): A 

In [8]:
from smexperiments.trial_component import TrialComponent

TrialComponent?

Init signature: TrialComponent(sagemaker_boto_client, **kwargs)
Docstring:     
This class represents a SageMaker trial component object.

A trial component is a stage in a trial.

Trial components are created automatically within the SageMaker runtime and may not be created directly.
To automatically associate trial components with a trial and experiment supply an experiment config when creating a
job. For example: https://docs.aws.amazon.com/sagemaker/latest/dg/API_CreateTrainingJob.html

Attributes:
    trial_component_name (str): The name of the trial component. Generated by SageMaker from the name of the
        source job with a suffix specific to the type of source job.
    trial_component_arn (str): The ARN of the trial component.
    display_name (str): The name of the trial component that will appear in UI, such as SageMaker Studio.
    source (obj): A TrialComponentSource object with a source_arn attribute.
    status (str): Status of the source job.
    start_time (datetime

In [9]:
training_data_bucket = f'sagemaker-sample-data-{boto_session.region_name}'
training_data_prefix = 'tensorflow/mnist'

training_data_uri = f's3://{training_data_bucket}/{training_data_prefix}'
training_data_uri

's3://sagemaker-sample-data-us-east-1/tensorflow/mnist'

In [10]:
boto_session.client('s3').list_objects(
Bucket = training_data_bucket,
Prefix = training_data_prefix)

{'ResponseMetadata': {'RequestId': 'GFWP5F9WKB3H6X4K',
  'HostId': '9/rD/ItGXwuh+qjYzr2iZpwU2DO+9NBAC7JAaxzqY4hord90cuW5bu9NR+mdsERKYvBXH+uBpqE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '9/rD/ItGXwuh+qjYzr2iZpwU2DO+9NBAC7JAaxzqY4hord90cuW5bu9NR+mdsERKYvBXH+uBpqE=',
   'x-amz-request-id': 'GFWP5F9WKB3H6X4K',
   'date': 'Sun, 13 Mar 2022 17:46:43 GMT',
   'x-amz-bucket-region': 'us-east-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Marker': '',
 'Contents': [{'Key': 'tensorflow/mnist/eval_data.npy',
   'LastModified': datetime.datetime(2019, 1, 24, 23, 36, 40, tzinfo=tzlocal()),
   'ETag': '"999d245abdf1f6f45d8b516ef04367e0-4"',
   'Size': 31360128,
   'StorageClass': 'STANDARD'},
  {'Key': 'tensorflow/mnist/eval_labels.npy',
   'LastModified': datetime.datetime(2019, 1, 24, 23, 36, 40, tzinfo=tzlocal()),
   'ETag': '"bf645d8a74fbd6d63977d0709bcb99c9"',
   'Size': 40